<a href="https://colab.research.google.com/github/OGsiji/Transrisk-Innovation/blob/main/Transrisk_Innovation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from accidents_montreal import get_accident_df
#from utils import init_spark
#from road_network import get_road_df
#from weather import add_weather_columns, extract_year_month_day
from pyspark.sql.functions import row_number, col, rank, avg, split, to_date, min

ModuleNotFoundError: ignored

In [ ]:

  
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import os
import re
from zipfile import ZipFile
from shutil import rmtree
from io import BytesIO
from bs4 import BeautifulSoup
#from pyspark.sql import Window
from pyspark.sql.functions import col, abs, hash, atan2, \
                                  sqrt, cos, sin, radians, \
                                  udf, monotonically_increasing_id, \
                                  concat, row_number
from pyspark.sql.types import StringType
from utils import raise_parquet_not_del_error
from workdir import workdir
from road_network_nids import unknow_file_included_nids


def get_road_df(spark, use_cache=True):
    fetch_road_network()
    return extract_road_segments_df(spark, use_cache)


def get_road_features_df(spark, road_df=None, use_cache=True):
    cache = workdir + "data/road-features.parquet"
    if os.path.isdir(cache) and use_cache:
        print('Skip extracting road features: already done')
        return spark.read.parquet(workdir+'data/road-features.parquet')

    road_df = road_df or get_road_df(spark)

    print('Extracting road features...')
    assign_street_type_udf = udf(assign_street_type, StringType())
    earth_diameter = 6371 * 2 * 1000  # in meters
    road_features = (road_df
                     .select('street_id', 'street_type', 'street_name',
                             'coord_lat', 'coord_long', 'center_lat',
                             'center_long')
                     .join(road_df.select(
                                'street_id',
                                col('coord_lat').alias('coord2_lat'),
                                col('coord_long').alias('coord2_long')),
                           'street_id')
                     .withColumn('distance_inter',
                                 distance_intermediate_formula(
                                    'coord_lat',
                                    'coord_long',
                                    'coord2_lat',
                                    'coord2_long'))
                     .withColumn('dist_measure', distance_measure())
                     .select('street_id', 'street_type', 'street_name',
                             'dist_measure', 'center_lat', 'center_long')
                     .groupBy('street_id', 'street_type', 'street_name',
                              'center_lat', 'center_long')
                     .max('dist_measure')
                     .withColumn('street_length',
                                 col('max(dist_measure)') * earth_diameter)
                     .select('street_id',
                             col('street_type').alias('street_level'),
                             'street_name',
                             'street_length',
                             'center_lat',
                             'center_long')
                     .withColumnRenamed('center_lat', 'loc_lat')
                     .withColumnRenamed('center_long', 'loc_long')
                     .withColumn('street_type',
                                 assign_street_type_udf(col('street_name')))
                     .drop('street_name'))

    if use_cache:
        road_features.write.parquet(cache)
    print('Extracting road features: done')
    return road_features.distinct()


def fetch_road_network():
    if not os.path.isdir(workdir + 'data'):
        os.mkdir(workdir + 'data')
    if os.path.isfile(workdir + 'data/road-network.lock'):
        print('Skip fetching road network: already downloaded')
        return
    print('Fetching road network...')
    url = 'http://ftp.maps.canada.ca/pub/nrcan_rncan'\
          '/vector/geobase_nrn_rrn/qc/kml_en/'
    if not os.path.isdir(workdir + 'data/road-network'):
        os.mkdir(workdir + 'data/road-network')
    files = \
        ['TNO_terrestre_du_TE_de_Montréal_6_60.kmz', 'Mont-Royal_4_58.kmz',
         "L'Île-Dorval_4_57.kmz", 'Montréal_4_58.kmz', 'Unknown_5_57.kmz',
         'Unknown_5_58.kmz', 'Unknown_4_55.kmz', 'Montréal_5_59.kmz',
         'Unknown_5_56.kmz', 'Montréal_4_55.kmz', "L'Île-Dorval_4_58.kmz",
         'Mont-Royal_5_59.kmz', 'Montréal_4_60.kmz', 'Montréal_4_56.kmz',
         'Montréal_6_60.kmz', 'Montréal-Ouest_4_58.kmz',
         'Senneville_4_56.kmz', 'Montréal_6_58.kmz', 'Unknown_4_59.kmz',
         'Montréal_4_57.kmz', 'Montréal_5_58.kmz', 'Montréal_7_60.kmz',
         'Unknown_6_60.kmz', 'Mont-Royal_5_58.kmz', 'Hampstead_4_58.kmz',
         'Montréal-Est_6_60.kmz', 'Sainte-Anne-de-Bellevue_4_56.kmz',
         'Hampstead_4_59.kmz', 'Dorval_4_57.kmz',
         'TNO_aquatique_du_TE_de_Montréal_5_60.kmz', 'Montréal_5_60.kmz',
         'Unknown_4_60.kmz', 'TNO_terrestre_du_TE_de_Montréal_5_60.kmz',
         'Montréal_5_56.kmz', 'Côte-Saint-Luc_5_58.kmz',
         'Dollard-Des_Ormeaux_5_57.kmz', 'Dorval_4_58.kmz',
         "Baie-D'Urfé_4_56.kmz", 'TNO_aquatique_du_TE_de_Montréal_5_58.kmz',
         'Senneville_4_55.kmz', 'Unknown_6_59.kmz', 'Beaconsfield_4_57.kmz',
         'Sainte-Anne-de-Bellevue_4_55.kmz', 'Montréal_5_57.kmz',
         'Westmount_4_59.kmz', 'Beaconsfield_4_56.kmz', 'Unknown_4_56.kmz',
         'Unknown_5_59.kmz', 'Unknown_4_57.kmz', 'Kirkland_4_57.kmz',
         'Westmount_5_59.kmz', 'Côte-Saint-Luc_4_58.kmz',
         'Pointe-Claire_4_57.kmz', 'Montréal_4_59.kmz', 'Unknown_5_60.kmz',
         'Côte-Saint-Luc_4_59.kmz', 'Unknown_4_58.kmz',
         'Dollard-Des_Ormeaux_4_56.kmz', 'Dorval_5_57.kmz',
         'Montréal-Ouest_4_59.kmz', 'Pointe-Claire_4_56.kmz',
         'Montréal-Est_6_59.kmz', 'Dollard-Des_Ormeaux_4_57.kmz',
         'Montréal_6_59.kmz']
    for file in files:
        urlretrieve(f'{url}{quote(file)}',
                    workdir + 'data/road-network/{0}'.format(file))
    open(workdir + 'data/road-network.lock', 'wb').close()
    print('Fetching road network done')


def get_kml_content(soup):
    ''' Function to extract kml file content and store relevant information
    into a pandas dataframe.
    Args:
        soup: File content extracted using beautiful soup
    '''
    rows = list()
    folders = soup.find_all('Folder')
    for folder in folders:
        street_type = folder.find('name').text
        placemarks = folder.find_all('Placemark')

        for placemark in placemarks:
            street_name = placemark.find('name').text
            center = placemark.MultiGeometry.Point.coordinates.text.split(',')
            coordinates_list = (placemark.MultiGeometry.LineString.coordinates
                                .text.split(' '))
            description = placemark.find('description').text
            nid = (re.search('<th>nid</th>\n<td>([a-f0-9]+)</td>', description)
                   .group(1))
            is_unknown = \
                re.search('<th>left_OfficialPlaceName</th>\n<td>Unknown</td>',
                          description) is not None
            if is_unknown and (nid not in unknow_file_included_nids):
                continue

            for coord in coordinates_list:
                coords = coord.split(',')
                if len(coords) > 1:
                    rows.append([
                            street_name,
                            street_type,
                            float(center[0]),
                            float(center[1]),
                            float(coords[0]),
                            float(coords[1]),
                            nid])
            # Add center of the street as a point of the street
            rows.append([
                    street_name,
                    street_type,
                    float(center[0]),
                    float(center[1]),
                    float(center[0]),
                    float(center[1]),
                    nid])
    return rows


def kml_extract_RDD(xml_file):
    ''' Function to extract the content of a kml input file and to store it
    into a csv output file.
    Args:
        xml_file_path: input kml file (kml is an xml file)
    '''
    soup = BeautifulSoup(xml_file, "lxml-xml")
    return get_kml_content(soup)


def get_road_segments_RDD(spark):
    def read_doc_from_zip_file(file):
        file_path = workdir + 'data/road-network/{0}'.format(file)
        return (BytesIO(ZipFile(file_path, 'r')
                .read('doc.kml')))

    return (spark.sparkContext
            .parallelize(os.listdir(workdir + 'data/road-network/'))
            .map(read_doc_from_zip_file))


def extract_road_segments_df(spark, use_cache=True):
    cache = workdir + 'data/road-network.parquet'
    if os.path.isdir(cache) and use_cache:
        print('Skip extraction of road network dataframe: already done,'
              ' reading from file')
        return spark.read.parquet(cache)

    print('Extracting road network dataframe...')
    cols = ['street_name', 'street_type', 'center_long', 'center_lat',
            'coord_long', 'coord_lat', 'nid']

    road_seg_df = (get_road_segments_RDD(spark)
                   .flatMap(kml_extract_RDD)
                   .toDF(cols))

    # Some specific road segments have the same nid
    w = Window.partitionBy('nid').orderBy('center_lat')
    street_ids = \
        (road_seg_df
         .select('nid', 'center_lat', 'center_long')
         .distinct()
         .select('center_lat',
                 'center_long',
                 concat('nid', row_number().over(w)).alias('street_id')))

    road_seg_df = (road_seg_df
                   .join(street_ids, ['center_lat', 'center_long'])
                   .drop('nid'))

    if use_cache:
        road_seg_df.write.parquet(cache)
    print('Extracting road network dataframe done')
    return road_seg_df


def distance_intermediate_formula(lat1, long1, lat2, long2):
    ''' Returns spark expression computing intermediate result
        to compute the distance between to GPS coordinates
        Source: https://www.movable-type.co.uk/scripts/latlong.html
    '''
    return (pow(sin(radians(col(lat1) - col(lat2))/2), 2)
            + (pow(sin(radians(col(long1) - col(long2))/2), 2)
               * cos(radians(col(lat1))) * cos(radians(col(lat2)))))


def distance_measure():
    return atan2(sqrt(col('distance_inter')),
                 sqrt(1-col('distance_inter')))


def assign_street_type(street_name):
    possible_keywords = street_name.split(' ')[0:1]
    possible_keywords = [pk.lower() for pk in possible_keywords]
    assignation = {
      'allée': ['allée'],
      'autoroute': ['autoroute'],
      'avenue': ['avenue'],
      'boulevard': ['boulevard'],
      'carré': ['carré'],
      'square': ['square'],
      'carref.': ['carref.'],
      'chemin': ['chemin'],
      'circle': ['circle', 'cercle'],
      'côte': ['côte'],
      'cours': ['cours'],
      'court': ['court'],
      'crescent': ['crescent', 'croissant'],
      'drive': ['drive'],
      'esplanade': ['esplanade'],
      'island': ['Île'],
      'impasse': ['impasse'],
      'lane': ['lane'],
      'lieu': ['lieu'],
      'montée': ['montée'],
      'park': ['parc', 'park'],
      'passage': ['passage'],
      'place': ['place'],
      'pont': ['pont'],
      'promenade': ['promenade'],
      'rang': ['rang'],
      'road': ['road', 'route'],
      'ruelle': ['ruelle'],
      'street': ['street', 'rue'],
      'terrasse': ['terrasse']
    }
    for street_type in assignation:
        for keyword in assignation[street_type]:
            if keyword in possible_keywords:
                return street_type



ModuleNotFoundError: ignored

In [ ]:



import time
import datetime

start = "01/01/" + str('2012')
end = "31/12/" + str('2018')
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%Y-%m-%d"), i))

In [ ]:
import pandas as pd

In [ ]:
Data = pd.DataFrame(dates, columns=('Date', 'hour'))

In [ ]:
Data.head()

,Date,hour
0,2012-01-02,0
1,2012-01-02,1
2,2012-01-02,2
3,2012-01-02,3
4,2012-01-02,4


In [ ]:
Data.shape

(61344, 2)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "drive/My Drive/"


Mounted at /content/drive/
/content/drive/My Drive


In [ ]:
data2=pd.read_csv('dataBIS correct.csv')

In [ ]:
data2.drop(columns=['DATE','CRASH TIME (HRS)'], inplace=True)

In [ ]:
data2.head()

,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
0,SUN,KM8-OOLO,NaN,OGB-OYO,CAR,COMM,05/02/2013 07:00,3,3.89100,7.39975
1,MON,KM8-FLYOVER,NaN,OGB-ILN,BUS,COMM.,4/21/2013 5:00,4,3.90820,7.42622
2,MON,KM24-IROKO,NaN,IBD-OYO,BUS,COMM.,4/27/2013 16:00,4,3.91593,7.51316
3,MON,KM2-SHARPCORNER,NaN,OGB-OYO,TRK,COMM.,06/05/2013 10:00,4,3.91466,7.59099
4,TUE,KM1-AJE IYA,NaN,OGB-OYO,TRL/ MC,COMM/ PVT,06/01/2013 11:00,5,3.92260,7.63765


i

In [ ]:
data2=pd.concat([Data,data2],axis=1)

In [ ]:
data2.head()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
0,2012-01-02,0,SUN,KM8-OOLO,NaN,OGB-OYO,CAR,COMM,05/02/2013 07:00,3.0,3.89100,7.39975
1,2012-01-02,1,MON,KM8-FLYOVER,NaN,OGB-ILN,BUS,COMM.,4/21/2013 5:00,4.0,3.90820,7.42622
2,2012-01-02,2,MON,KM24-IROKO,NaN,IBD-OYO,BUS,COMM.,4/27/2013 16:00,4.0,3.91593,7.51316
3,2012-01-02,3,MON,KM2-SHARPCORNER,NaN,OGB-OYO,TRK,COMM.,06/05/2013 10:00,4.0,3.91466,7.59099
4,2012-01-02,4,TUE,KM1-AJE IYA,NaN,OGB-OYO,TRL/ MC,COMM/ PVT,06/01/2013 11:00,5.0,3.92260,7.63765


In [ ]:
import random
import numpy as np

#data2["DAY"].fillna(lambda x: random.choice(data2[data2["DAY"] != np.nan]["DAY"]), inplace =True)

In [ ]:
data2["DAY"] = data2["DAY"].apply(lambda x: x.fillna(random.choice(x.dropna())))

AttributeError: ignored

In [ ]:
def na_randomfill(series):
    na_mask = pd.isnull(series)   # boolean mask for null values
    n_null = na_mask.sum()        # number of nulls in the Series
    
    if n_null == 0:
        return series             # if there are no nulls, no need to resample
    
    # Randomly sample the non-null values from our series
    #  only sample this Series as many times as we have nulls 
    fill_values = series[~na_mask].sample(n=n_null, replace=True, random_state=0)

    # This ensures our new values will replace NaNs in the correct locations
    fill_values.index = series.index[na_mask]
    
    return series.fillna(fill_values) 

In [ ]:
data23 = data2.apply(na_randomfill)

In [ ]:
data23['PROBABLE CAUSE(S) OF RTC'].nunique()

103

In [ ]:
data23[data23['LOCATION'] == 'KM7.5-BABA IBADAN VILLAGE']

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
577,2012-01-26,1,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
978,2012-02-11,18,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
2139,2012-03-31,3,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
2869,2012-04-30,13,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
3472,2012-05-25,16,TUE,KM7.5-BABA IBADAN VILLAGE,TBT,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
...,...,...,...,...,...,...,...,...,...,...,...,...
59605,2018-10-20,13,TUE,KM7.5-BABA IBADAN VILLAGE,MDV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
59981,2018-11-05,5,TUE,KM7.5-BABA IBADAN VILLAGE,RTV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
60070,2018-11-08,22,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
60290,2018-11-18,2,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694


In [ ]:
data23.tail()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
61339,2018-12-31,19,TUE,KM7.5-BABA IBADAN VILLAGE,"SPV,WOV",OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.69400
61340,2018-12-31,20,MON,KM4.5- CARETAKER,WOV,OGB-OYO,TNK/ MC/ MC,COMM/ COMM/ PVT,12/02/2018 06:00,9.0,3.90930,7.55428
61341,2018-12-31,21,THU,KM6-ARINKINKIN AREA,"DGD,LSV",OGB-OYO,MC/ PICK UP,PVT/ GOVT,12/27/2014 20:00,11.0,4.36135,8.30588
61342,2018-12-31,22,SAT,KM7-DEPEER LIFE AREA,SPV,OYO-IBD,SUV,PVT,06/04/2018 22:00,3.0,4.39034,8.31515
61343,2018-12-31,23,WED,KM1-OYO EAST LG,SPV,OYO-OGB,TRL/ BUS/ MC,COMM/ COMM/ PVT,10/20/2016 5:00,8.0,4.23288,8.12462


In [ ]:
data23.tail()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y,3.0,4.0,5.0,11.0,2.0,8.0,9.0,12.0,1.0,7.0,10.0,6.0
61339,2018-12-31,19,TUE,KM7.5-BABA IBADAN VILLAGE,"SPV,WOV",OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.69400,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61340,2018-12-31,20,MON,KM4.5- CARETAKER,WOV,OGB-OYO,TNK/ MC/ MC,COMM/ COMM/ PVT,12/02/2018 06:00,9.0,3.90930,7.55428,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61341,2018-12-31,21,THU,KM6-ARINKINKIN AREA,"DGD,LSV",OGB-OYO,MC/ PICK UP,PVT/ GOVT,12/27/2014 20:00,11.0,4.36135,8.30588,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61342,2018-12-31,22,SAT,KM7-DEPEER LIFE AREA,SPV,OYO-IBD,SUV,PVT,06/04/2018 22:00,3.0,4.39034,8.31515,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61343,2018-12-31,23,WED,KM1-OYO EAST LG,SPV,OYO-OGB,TRL/ BUS/ MC,COMM/ COMM/ PVT,10/20/2016 5:00,8.0,4.23288,8.12462,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [ ]:
import numpy as np
def fill_with_random(data2, DAY):
    '''Fill `df2`'s column with name `column` with random data based on non-NaN data from `column`'''
    data2 = data2.copy()
    data2['DAY'] = data2['DAY'].apply(lambda x: np.random.choice(data2['DAY'].dropna().values) if np.isnan(x) else x)
    return data2

In [ ]:
data2.apply(fill_with_random(data2, 'DAY'))

TypeError: ignored

In [ ]:
len(Data)

61344

In [ ]:
df = pd.DataFrame({'empty' : []})

for row in df:
    new_df = pd.concat(data2, axis=1)

TypeError: ignored

In [ ]:
Real_Data = pd.concat(['Data','data2'],axis=1)

TypeError: ignored